In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [6]:
from typing import Any, Dict

import boto3
import yaml
import json

from core.event_processor import EventProcessor
from core.task_publisher import TaskPublisher

from services.messages_dynamodb import MessagesDynamodbService
from services.messages_whatsapp import MessagesWhatsappService
from integrations.messages import MessagesIntegration

In [7]:
_sqs = None
_processor = None
_queue_url = None

_s3 = None
_bucket = None
_verify_token = None

_history_service = None
_whatsapp_service = None
_messages_integration = None

In [8]:
with open("../config/tenants/dev.yaml", "r", encoding="utf-8") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {}) or {}
dynamodb = boto3.resource( "dynamodb", region_name=dynamodb_config.get("region"))

messages_table = dynamodb.Table(dynamodb_config["messages_table"])
processes_table = dynamodb.Table(dynamodb_config["processes_table"])

sqs_config = tenant_config.get("sqs", {}) or {}
_sqs = boto3.client("sqs", region_name=sqs_config.get("region"))

_queue_url = (sqs_config.get("tasks_url"))

_processor = EventProcessor(
    processes_table=processes_table,
    task_publisher=TaskPublisher(_sqs, _queue_url),
)

print("INIT: processor ready")

whatsapp_config = tenant_config.get("whatsapp", {}) or {}
_verify_token = whatsapp_config.get("verify_token")

s3_config = tenant_config.get("s3", {}) or {}
_bucket = s3_config.get("bucket")
_s3 = boto3.client("s3", region_name=s3_config.get("region"))

_history_service = MessagesDynamodbService(messages_table)
_whatsapp_service = MessagesWhatsappService(tenant_config=tenant_config)
_messages_integration = MessagesIntegration(_history_service, _whatsapp_service)

INIT: processor ready


In [9]:
event = {
    "identity": "whatsapp:525571969848",
    "message_key": "2025-12-18T19:14:52#in#wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDJBMTEyODQzMEJDMjU5NTEyRDRDAA==",
    "channel": "whatsapp",
    "content": {
        "text": "En qué nació borges"
    },
    "direction": "in",
    "message_type": "text",
    "payload": {
        "entry": [
            {
                "changes": [
                    {
                        "field": "messages",
                        "value": {
                            "contacts": [
                                {
                                    "profile": {
                                        "name": "Juan Ignacio Fernández Larriera"
                                    },
                                    "wa_id": "5215571969848"
                                }
                            ],
                            "messages": [
                                {
                                    "from": "5215571969848",
                                    "id": "wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDJBMTEyODQzMEJDMjU5NTEyRDRDAA==",
                                    "text": {
                                        "body": "En qué nació borges"
                                    },
                                    "timestamp": "1766085292",
                                    "type": "text"
                                }
                            ],
                            "messaging_product": "whatsapp",
                            "metadata": {
                                "display_phone_number": "15556077069",
                                "phone_number_id": "328246003707574"
                            }
                        }
                    }
                ],
                "id": "369148326271872"
            }
        ],
        "object": "whatsapp_business_account"
    },
    "timestamp_epoch": 1766085292,
    "timestamp_iso": "2025-12-18T19:14:52"
}

In [10]:
def handler(processor, event):
    identity = event.get("identity") or ""
    identity_part = identity.split(":", 1)[-1] if ":" in identity else identity

    content = dict(event.get("content") or {})
    message_type = event.get("message_type")

    msg_id = event.get("msg_id")
    timestamp_iso = event.get("timestamp_iso")
    timestamp_epoch = event.get("timestamp_epoch")

    processor.process(
        process_type="WHATSAPP_CONVERSATION",
        event="WHATSAPP_MESSAGE_RECEIVED",
        context={"identity": identity},
        payload={
            "identity": identity,
            "phone": identity_part,
            "msg_id": msg_id,
            "message_type": message_type,
            "content": content,
            "timestamp_iso": timestamp_iso,
            "timestamp_epoch": timestamp_epoch,
        },
    )

    if message_type == "document":
        document_id = content.get("media_id")
        if document_id:
            processor.process(
                process_type="WHATSAPP_DOCUMENT_PIPELINE",
                event="DOCUMENT_RECEIVED",
                context={"msg_id": msg_id, "document_id": document_id},
                payload={
                    "identity": identity,
                    "phone": identity_part,
                    "msg_id": msg_id,
                    "document_id": document_id,
                    "timestamp_iso": timestamp_iso,
                    "timestamp_epoch": timestamp_epoch,
                    "document": content,
                },
            )

In [11]:
def handle_webhook(event):
    try:
        payload = event.get("payload")
    except json.JSONDecodeError:
        print("INVALID_JSON:", body_raw)
        return {"statusCode": 400, "body": "Invalid JSON"}

    events = _messages_integration.parse_incoming_payload(payload)

    for ev in events:
        identity = ev.get("identity") or ""
        identity_part = identity.split(":", 1)[-1] if ":" in identity else identity
        content = dict(ev.get("content") or {})
        timestamp_epoch = ev.get("timestamp_epoch")
        message_type = ev.get("message_type")
        ext = "bin"

        if message_type in ["image", "video", "audio", "document"]:
            media_id = content.get("media_id")
            if media_id:
                meta_result = _whatsapp_service.get_media_metadata(media_id)
                if meta_result.get("status") == "ok":
                    meta = meta_result.get("data") or {}
                    download_url = meta.get("url")
                    mime_type = meta.get("mime_type") or content.get("mime_type")
                    filename = content.get("filename")

                    if filename and "." in filename:
                        ext = filename.rsplit(".", 1)[-1].lower()
                    elif mime_type and "/" in mime_type:
                        ext = mime_type.split("/")[-1].lower()

                    if download_url:
                        download_result = _whatsapp_service.download_media(download_url)
                        if download_result.get("status") == "ok":
                            media_bytes = download_result.get("content") or b""

                            media_key = f"whatsapp_media/{identity_part}/{timestamp_epoch}_{media_id}.{ext}"

                            _s3.put_object(
                                Bucket=_bucket,
                                Key=media_key,
                                Body=media_bytes,
                            )

                            content["media_s3_key"] = media_key
                            if mime_type:
                                content["mime_type"] = mime_type
                            if filename:
                                content["filename"] = filename
                        else:
                            print("MEDIA_DOWNLOAD_ERROR", media_id, download_result)
                else:
                    print("MEDIA_META_ERROR", media_id, meta_result)
        
        _history_service.save_message(
            identity=identity,
            direction="in",
            channel="whatsapp",
            message_type=message_type,
            timestamp_iso=ev.get("timestamp_iso"),
            timestamp_epoch=timestamp_epoch,
            content=content,
            payload=ev.get("raw_payload"),
            msg_id=ev.get("msg_id")
        )
        try:
            handler(_processor, ev)
        except Exception as e:
            print("HANDLER_RUNTIME_ERROR:", str(e))

    return {"statusCode": 200, "body": "EVENT_RECEIVED"}

In [12]:
response = handle_webhook(event)

1113124321
📤 Publishing task 55a13d56-a3d3-4bc0-9708-da6a61cbd66d → ANSWER_INCOMING_WHATSAPP_MESSAGE delay=60s


In [13]:
response

{'statusCode': 200, 'body': 'EVENT_RECEIVED'}